# Collect 16S sequences from NCBI

In [1]:
# https://biopython.org/docs/latest/Tutorial/chapter_entrez.html#sec-entrez-webenv
# Börja med att importera rätt paket, och lägg in mejladress för att NCBI ska veta vem man är (idk)
# Kräver att Biopython är installerat

import pandas as pd
from Bio import Entrez
Entrez.email = 'clara.nordquist.1217@student.uu.se'

In [2]:
# Läsa in vilka arter som vi ska hämta sekvenser för
organism_dataset = pd.read_csv('/Users/claranordquist/Documents/Universitetet/HT24/Tillämpad bioinformatik/Applied-bioinformatics/Collect sequences/Test data/Species_to_identify_species_and_genus.csv', names=['species'])
organisms = [a for a in organism_dataset.iloc[:, 0]]

In [3]:
# Definiera hur många sekvenser som ska hämtas för varje art, samt namnet på outputfilen
max_sequences = 2
output = open('All_given_species_genus.fasta', 'w')

# En lista för alla arter som ej finns i databasen
not_found = []

# För varje organism i listan, sök i NCBI (RefSeq) efter 16S rRNA-gener med längd mellan 1400 och 1650 bp
# Spara de [max_sequences] första träffarna i fastaformat, i den angivna outputfilen
# Om organismens ej finns i databasen, lägg namnen i en lista
for organism in organisms: 
    search_term = f'''
    ({organism}[Organism] AND 16S[All Fields] 
    AND rRNA[All Fields]) AND bacteria[filter] 
    AND (refseq[filter] AND ("1400"[SLEN] : "1650"[SLEN]))
    '''
    stream = Entrez.esearch(db = 'nucleotide', term = search_term, usehistory = 'y', idtype = 'acc')
    search_results = Entrez.read(stream)
    stream.close()
    acc_list = search_results['IdList']
    webenv = search_results['WebEnv']
    query_key = search_results['QueryKey']
    
    stream = Entrez.efetch(
    db = 'nucleotide', rettype = 'fasta', retmode = 'text', retmax = 2, webenv = webenv, query_key = query_key, idtype = 'acc')
    data = stream.read()
    
    if type(data) == str:
        output.write(data)
    else:
        not_found.append(organism)
output.close()